In [1]:
#Reference: https://www.kaggle.com/danrusei/10-monkey-keras-transfer-learning-resnet50

import numpy as np 
import pandas as pd 

from tensorflow.contrib.keras.api.keras.callbacks import Callback
from tensorflow.contrib.keras.api.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras import backend
from keras.optimizers import Adam
from glob import glob
import os

Using TensorFlow backend.


In [ ]:
training_dir = "/Users/vineevineela/Documents/Semester-4/CMPE-256/Project/dataset/training"
test_dir = "/Users/vineevineela/Documents/Semester-4/CMPE-256/Project/dataset/validation"

In [ ]:
training_dir

In [ ]:
# useful for getting number of files
train_count = glob(training_dir + '/*/*.jp*g')
test_count = glob(test_dir + '/*/*.jp*g')

In [ ]:
len(test_dir)

In [ ]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print(num_classes)

In [ ]:
from keras.applications import DenseNet201
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
import keras
my_new_model = Sequential()

my_new_model.add(DenseNet201(include_top=False, pooling='avg', weights='imagenet'))
my_new_model.add(Dense(4096,activation='relu'))
my_new_model.add(keras.layers.Dropout(0.5))

my_new_model.add(Dense(num_classes, activation='softmax'))

# my_new_model.layers[0].trainable = False
for layers in my_new_model.layers[-2:]:
    layers.trainable=True


In [ ]:
my_new_model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
input_size = (64,64)
batch_size = 32

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip = True,
                                    preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    preprocessing_function=preprocess_input)


training_generator = training_datagen.flow_from_directory(training_dir, target_size = input_size, batch_size = batch_size, class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size = input_size, batch_size = batch_size, class_mode= 'categorical')


In [ ]:
import keras
opt = keras.optimizers.Adam(lr=1e-3, decay=1e-6) 
my_new_model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
test_generator.class_indices

In [ ]:
model_info = my_new_model.fit_generator(training_generator,
                         steps_per_epoch=7721/batch_size,
                         epochs=2,
                         validation_data=test_generator,
                         validation_steps=856/batch_size,
                         workers=12)